# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*

## Практикум 2.2. Формирование ссылок и парсинг полученных страниц

**Полезное напоминание 1.** Чтобы получать списки с нужной структурой, давайте вспомним, в чем различие методов `.append()` и `.extend()` для добавления элементов списка.

In [1]:
L = []
L.append(3)
L.append(5)
L.append([3, 8, 10])
print(L)

[3, 5, [3, 8, 10]]


In [2]:
K = []
# K.extend(3)

In [3]:
K.extend([3])
K.extend([5])
K.extend([3, 8, 10])
print(K)

[3, 5, 3, 8, 10]


Чтобы понять, как это может быть полезно при сборе данных, представьте, что нам нужно собрать однотипные данные не с одной страницы, а сразу с нескольких. Какой способ предпочтительнее, например, если мы хотим получить список ссылок на все товары, которые есть на первой, второй и третьей странице каталога?

**Полезное напоминание 2.** Чтобы объединить элементы списков попарно, можно воспользоваться функцией `zip()`:

In [4]:
zip(["A", "B", "C"], [10, 20, 30])

In [5]:
print(*zip(["A", "B", "C"], [10, 20, 30]))

('A', 10) ('B', 20) ('C', 30)


По перечню пар можно двигаться в рамках одного цикла `for`, присваивая названия первому и второму элементу пары соответственно:

In [6]:
pairs = zip(["A", "B", "C"], [10, 20, 30])

for letter, number in pairs:
    print(letter, "has", number, "apples")

A has 10 apples
B has 20 apples
C has 30 apples


Чтобы понять, как это может быть полезно при сборе данных, представьте, что нам нужно сформировать ссылки, которые содержат даты. Для перебора дней в каждом месяце будет полезно иметь набор пар «месяц-число дней в месяце».

**Полезное напоминание 3.** Вспомним про форматирование строк через f-строки (*f-strings*, *formatted string literals*):

In [7]:
name = "Alla"
surname = "Turner"
print(f"Hello, {name} {surname}!")

Hello, Alla Turner!


In [8]:
N = 5
print(f"Let's count from 1 to {5}!")
for i in range(1, N+1):
    print(f"{i}")

Let's count from 1 to 5!
1
2
3
4
5


Чтобы понять, как это может быть полезно при сборе данных, представьте, что в однотипные по структуре ссылки нам необходимо подставлять какие-то слова или числа во избежание «ручного» формирования списка ссылок.

Перейдем к содержательным задачам. Импортируем модуль `requests`, функцию `BeautifulSoup`, а также библиотеку `pandas` для дальнейшей обработки данных:

In [9]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Сюжет 1. Форматирование строк и подстановка текста в ссылки

Зайдем на [сайт](https://www.chitai-gorod.ru) книжного магазина «Читай-город» и посмотрим, что происходит с ссылкой в адресной строке браузера, когда мы отправляем запрос для поиска товара. Давайте введем слово `python` и скопируем ссылку из адресной строки.

In [10]:
# сама ссылка для python – будем с ней работать
u = "https://www.chitai-gorod.ru/search?phrase=python"

Пример общего варианта с подстановкой любого запроса из переменной `q` (в том числе на случай запроса из нескольких слов):

In [11]:
q = "python для детей"

# заменяем пробелы на %20 – см сайт магазина
q_upd = q.replace(" ", "%20")

# подставляем q_upd в ссылку
q_link = f"https://www.chitai-gorod.ru/search?phrase={q_upd}"
print(q_link)

https://www.chitai-gorod.ru/search?phrase=python%20для%20детей


### Задача 1

Напишите код, который отправляет запрос к странице по ссылке выше, забирает HTML-код, находит все «карточки» товаров на первой странице результатов и сохраняет их в виде списка объектов типа `BeautifulSoup`.

In [12]:
soup = BeautifulSoup(requests.get(u).text)
arts = soup.find_all("article")

In [13]:
# пример одного элемента из arts,
# объект BeautifulSoup – сырой фрагмент кода HTML

a = arts[0]
#print(a)

### Задача 2

Напишите функцию, которая принимает на вход объект типа `BeautifulSoup` с «карточкой» товара (как в списке выше) и возвращает список, состоящий из: 

* названия товара;
* ссылки на страницу товара;
* цены товара.

Примените функцию ко всем элементам списка из предыдущей задачи и проверьте, что все работает.

In [14]:
# на входе: сырой фрагмент кода с карточкой товара (как в списке выше)
# на выходе: список с элементами из этой карточки в чистом виде

def get_item(a):
    name = a.get("data-chg-product-name")
    price = a.get("data-chg-product-price")
    url = a.find("a").get("href")
    full_url = "https://www.chitai-gorod.ru" + url
    return [name, price, full_url]

In [15]:
L = [get_item(a) for a in arts]
# первые три элемента для примера
L[0:3]

[['Python. Полное руководство',
  '906',
  'https://www.chitai-gorod.ru/product/python-polnoe-rukovodstvo-2893579'],
 ['Справочник PYTHON. Кратко, быстро, под рукой',
  '528',
  'https://www.chitai-gorod.ru/product/spravochnik-python-kratko-bystro-pod-rukoy-2854644'],
 ['Python с нуля',
  '1799',
  'https://www.chitai-gorod.ru/product/python-s-nulya-3028159']]

### Задача 3

Полистайте страницы с результатами поиска в браузере и посмотрите, как изменяется ссылка в адресной строке при переходе к следующей странице с результатами.

Напишите код, который подставляет в ссылку номера страниц 1-3 и возвращает список с «карточками» всех товаров на этих странице (как в задаче 1, только теперь больше товаров). Примените функцию из задачи 2 ко всем элементам полученного списка и преобразуйте полученный результат в датафрейм pandas.

In [16]:
all_pages = []
for i in range(1, 4):
    link = f"https://www.chitai-gorod.ru/search?phrase=python&page={i}"
    s = BeautifulSoup(requests.get(link).text)
    one_page = s.find_all("article")
    all_pages.extend(one_page)

In [17]:
print(len(all_pages)) # 48 * 3 = 144 товара

144


### Сюжет 2. Форматирование строк и подстановка чисел в ссылки

В практикуме 2.1 мы написали универсальную функцию для сайта [nplus1.ru](https://nplus1.ru), которая принимает на вход ссылку на новость, а возвращает кортеж с текстом новости и другими ее атрибутами. Остается вопрос: а как быть, если мы хотим собрать новости за конкретный период? Не на всех сайтах это возможно сделать удобным образом, однако в данном случае на сайте есть удобные разделы по датам. Например, откроем новости за 27 января 2025 года: https://nplus1.ru/news/2025/01/27.

Если мы посмотрим на страницу по ссылке выше, заметим, что на ней хранятся обычные ссылки на новости, как их собирать, мы уже знаем. Поэтому давайте поймем, как сформировать список ссылок с датами внутри.

### Задача 1

Получите список ссылок на новости за январь 2025 года (с 1 января по 31 января включительно). 

In [18]:
# работа .zfill()
# в начало строки дописываются 0, чтобы получить нужное число символов

print("1".zfill(2))
print("1".zfill(3))
print("123".zfill(3))

01
001
123


In [19]:
# незабываем превратить i в строку,
# .zfill() с числоами не работает

jan = []
for i in range(1, 32):
    day = str(i).zfill(2)
    url = f"https://nplus1.ru/news/2025/01/{day}"
    jan.append(url)
    
print(jan)

['https://nplus1.ru/news/2025/01/01', 'https://nplus1.ru/news/2025/01/02', 'https://nplus1.ru/news/2025/01/03', 'https://nplus1.ru/news/2025/01/04', 'https://nplus1.ru/news/2025/01/05', 'https://nplus1.ru/news/2025/01/06', 'https://nplus1.ru/news/2025/01/07', 'https://nplus1.ru/news/2025/01/08', 'https://nplus1.ru/news/2025/01/09', 'https://nplus1.ru/news/2025/01/10', 'https://nplus1.ru/news/2025/01/11', 'https://nplus1.ru/news/2025/01/12', 'https://nplus1.ru/news/2025/01/13', 'https://nplus1.ru/news/2025/01/14', 'https://nplus1.ru/news/2025/01/15', 'https://nplus1.ru/news/2025/01/16', 'https://nplus1.ru/news/2025/01/17', 'https://nplus1.ru/news/2025/01/18', 'https://nplus1.ru/news/2025/01/19', 'https://nplus1.ru/news/2025/01/20', 'https://nplus1.ru/news/2025/01/21', 'https://nplus1.ru/news/2025/01/22', 'https://nplus1.ru/news/2025/01/23', 'https://nplus1.ru/news/2025/01/24', 'https://nplus1.ru/news/2025/01/25', 'https://nplus1.ru/news/2025/01/26', 'https://nplus1.ru/news/2025/01/27', 

### Задача 2

Напишите код, который сформирует список ссылок на даты за весь 2024 год (31 ссылка на даты января, 29 ссылок на даты февраля и так далее).

**Подсказка:** начните со списка с номерами месяцев и списка с числом дней в них.

In [20]:
# месяцы
months = range(1, 13)
print(*months)

# дни в каждом месяце
ndays = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
print(*ndays)

# пары
print(*zip(months, ndays))

1 2 3 4 5 6 7 8 9 10 11 12
31 29 31 30 31 30 31 31 30 31 30 31
(1, 31) (2, 29) (3, 31) (4, 30) (5, 31) (6, 30) (7, 31) (8, 31) (9, 30) (10, 31) (11, 30) (12, 31)


In [21]:
year = []
for mon, n in zip(months, ndays):
    for date in range(1, n + 1):
        link_date = f"https://nplus1.ru/news/2024/{str(mon).zfill(2)}/{str(date).zfill(2)}"
        year.append(link_date)
print(len(year)) # 366 ссылок, високосный год

366


### Задача 3

Перейдите по [ссылке](https://nplus1.ru/news/2025/01/03) на страницу с новостями за 3 января 2025 года. Сравните полный исходный код страницы и код, который отображается в инструментах разработчика. Подумайте, каким образом в данном случае забирать все ссылки на новости за конкретную дату.

In [22]:
# в исходном коде нет ссылок на новости с тэгом <a>,
# но они есть где-то в блоке кода JavaScript с тэгом <script>
# поймем, в каком из элементов <script> эта информация,
# заберем текст и перейдем к практикуму 3

url = "https://nplus1.ru/news/2025/01/03"
soup = BeautifulSoup(requests.get(url).text)
js = soup.find_all("script")[8].text
# print(js)

**NB.** Если после исполнения кода выше в `js` пустая строка с текстом – замените `.text` на `.next`, это особенность другой версии `bs4`:

    soup.find_all("script")[8].next
    
Так как формально внутри `<script>` не текст, а код JavaScript, текст через атрибут `.text` в более новой версии `bs4` не находится (более строгое отношение к типам). А атрибут `.next` возвращает содержимое, следующее за тэгом, что в нашем случае – ровно то, что нужно.